# Generate catalog of stars using Hipparcos-2 and Gaia data for stars brighter than 15 mag in $V$-band

Everything that is not J2016 are propagated forward to J2016.

To get Gaia data, the ADQL query used is as follow:

```sql
with x as
(
	SELECT G.source_id, H2.original_ext_source_id as hip, G.ra, G.dec, G.parallax, G.parallax_over_error, G.pmra, G.pmdec, G.phot_g_mean_mag, G.bp_rp, G.radial_velocity, G.radial_velocity_error, G.astrometric_params_solved, G.ruwe, G.rv_expected_sig_to_noise, G.phot_g_mean_mag - (0.01426 * POWER(G.bp_rp, 3) - 0.2156 * POWER(G.bp_rp, 2) + 0.01424 * POWER(G.bp_rp, 1) - 0.02704) as v_mag 
    FROM gaiadr3.gaia_source AS G
	LEFT JOIN gaiadr3.hipparcos2_best_neighbour AS H2 ON H2.source_id = G.source_id
	WHERE G.phot_g_mean_mag IS NOT NULL OR H2.source_id IS NOT NULL
)
SELECT * 
FROM x
WHERE (bp_rp IS NOT NULL AND v_mag <= 15.5) OR (bp_rp IS NULL AND phot_g_mean_mag <= 15.5)
```

correspond to file named `1733546104637O-result.fits` used in here (You will get a different filename if you do the same query yourself)

`B-V` color are computed from synthetic photometry with Gaia low-res spectra while $V$ simply estimated with Gaia $G$ and $G_\mathrm{bp} - G_\mathrm{rp}$

## Parse the result from SIMBAD

In [ ]:
import pathlib
import struct
import warnings

import numpy as np
import pandas as pd
import trimesh
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
from astroquery.gaia import Gaia

from py.gaia import apply_space_motion, gbprp_to_bv
from py.geodesic import GeodesicGrid, radec2xyz

# paths to the data
simbad_base_path = pathlib.Path("./simbad_query_results")
gaia_base_path = pathlib.Path("./gaia_query_results")
starcatalog_base_path = pathlib.Path("./star_catalogs")
synth_phot_path = gaia_base_path / "Gaia_XP_JKC.csv"
gaia_data_path = gaia_base_path / "1733546104637O-result.fits"

# Read the data
simbad_df = pd.read_csv(
    simbad_base_path / "hip_processed_with_binary.dat", low_memory=False
)
# insert default astrometry epoch to J2000 beacuse that is what SIMBAD uses
# if has gaia source_id, we will use Gaia DR3 astrometry instead
simbad_df["epoch"] = 2000.0

# delete HIP 55987B because it is way too dim
simbad_df = simbad_df[~((simbad_df["hip"] == 55987) & (simbad_df["componentid"] == 2))].reset_index(drop=True)
# add HIP 81693 B component
idx = (simbad_df["source_id"] == 1312665361414730624)
if np.sum(idx) == 0:
    # add the B component
    simbad_df.loc[len(simbad_df)] = {
            "hip": 81693,
            "componentid": 2,
            "source_id": 1312665361414730624,
            "plx_value": 93.32,
            "plx_err": 0.47,
            "epoch": 2000.0,
        }
# add HIP 84012 B component
idx = (simbad_df["source_id"] == 4136366183978856960)
if np.sum(idx) == 0:
    # add the B component
    simbad_df.loc[len(simbad_df)] = {
            "hip": 84012,
            "componentid": 2,
            "source_id": 4136366183978856960,
            "plx_value": 36.91,
            "plx_err": 0.80,
            "epoch": 2000.0,
        }
# add HIP 36850 B component
idx = (simbad_df["source_id"] == 892348694913501952)
if np.sum(idx) == 0:
    # add the B component
    simbad_df.loc[len(simbad_df)] = {
            "hip": 36850,
            "componentid": 2,
            "source_id": 892348694913501952,
            "plx_value": 66.356,
            "plx_err": 0.041,
            "epoch": 2000.0,
        }
# add HIP 84345 B component
idx = (simbad_df["source_id"] == 4541877995116193408)
if np.sum(idx) == 1:  # it exists as a single stars, so add the B component
    # change the componentid to 2
    simbad_df.loc[idx, "componentid"] = 2

idx = (simbad_df["hip"] == 84345) & ((simbad_df["componentid"] == 0) | (simbad_df["componentid"] == 1))
if np.sum(idx) == 0:
    # add the A component
    simbad_df.loc[len(simbad_df)] = {
            "ra": 258.661886421,
            "dec": 14.390371047,
            "hip": 84345,
            "source_id": 0,
            "componentid": 1,
            "plx_value": 9.08,
            "plx_err": 1.32,
            "pmra": -17.0,
            "pmdec": 47.0,
            "rvz_radvel": -33.1,
            "rvz_err": 0.9,
            "sp_type": "M5Ib-II",
            "B": 4.67,
            "V": 3.33,
            "epoch": 2000.0,
        }

# read synthetic photometry
synth_phot_df = pd.read_csv(gaia_base_path / "Gaia_XP_JKC.csv")

# SIMBAD matching is conservative, we can use Gaia best neighbor to match the rest
gaia_t = Table.read(gaia_data_path, format="fits")
gaia_t = gaia_t.to_pandas()

# if there are source_id which most HIP stars do, use astrometry from Gaia DR3 even if RA/DEC is not NaN
# because Gaia DR3 is much better
matched_source_id, idx1, idx2 = np.intersect1d(
    gaia_t["source_id"].values, simbad_df["source_id"].values, return_indices=True
)
# put the astrometry from Gaia DR3 to HIP table. Every stars in Gaia have RA and DEC, so we can safely put it in
simbad_df.loc[idx2, "ra"] = gaia_t.loc[idx1, "ra"].values
simbad_df.loc[idx2, "dec"] = gaia_t.loc[idx1, "dec"].values
# not every stars in Gaia have other astrometry, especially those stars dim enough to be
# observed but bright enough to not have astrometry besides RA and DEC
valid_parallax_idx = ~np.isnan(gaia_t.loc[idx1, "parallax"].values)
simbad_df.loc[idx2[valid_parallax_idx], "plx_value"] = gaia_t.loc[
    idx1[valid_parallax_idx], "parallax"
].values
simbad_df.loc[idx2[valid_parallax_idx], "plx_err"] = (
    gaia_t.loc[idx1[valid_parallax_idx], "parallax"].values
    / gaia_t.loc[idx1[valid_parallax_idx], "parallax_over_error"].values
)
valid_pm_idx = ~np.isnan(gaia_t.loc[idx1, "pmra"].values) & ~np.isnan(
    gaia_t.loc[idx1, "pmdec"].values
)
simbad_df.loc[idx2[valid_pm_idx], "pmra"] = gaia_t.loc[
    idx1[valid_pm_idx], "pmra"
].values
simbad_df.loc[idx2[valid_pm_idx], "pmdec"] = gaia_t.loc[
    idx1[valid_pm_idx], "pmdec"
].values
simbad_df.loc[idx2, "epoch"] = (
    2016.0  # gaia DR3 is at J2016.0 (because RA/DEC is replaced, even if not PMRA/PMDEC)
)

# Gaia DR3 5236791746461774080 is HIP 55987A, but SIMBAD has no source_id for it (also 15ish mag, so outside of this Gaia Catalog range)
idx = (simbad_df["hip"] == 55987) & (simbad_df["componentid"] == 1)
simbad_df.loc[idx, "source_id"] = 5236791746461774080
simbad_df.loc[idx, "G"] = 15.615307
simbad_df.loc[idx, "B"] = 16.615616
simbad_df.loc[idx, "V"] = 15.843829
simbad_df.loc[idx, "ra"] = 172.11765151979628
simbad_df.loc[idx, "dec"] = -66.48608788896621
simbad_df.loc[idx, "pmra"] = -8.85141903479891
simbad_df.loc[idx, "pndec"] = 0.3523543478864454
simbad_df.loc[idx, "plx_value"] = 0.6125993002975423
simbad_df.loc[idx, "plx_err"] = 0.0275
simbad_df.loc[idx, "radial_velocity"] = 0.0
simbad_df.loc[idx, "radial_velocity_error"] = 0.0
simbad_df.loc[idx, "epoch"] = 2016.0

# Gaia DR3 1312665361415345920 is HIP 81693A, but SIMBAD has no source_id for it
idx = ((simbad_df["hip"] == 81693) & (simbad_df["componentid"] == 0))
assert np.sum(idx) == 1, "There should be only one HIP 81693"
simbad_df.loc[idx, "source_id"] = 1312665361415345920
simbad_df.loc[idx, "componentid"] = 1  # because it is A component
# HIP 84012 to HIP 84012 B
idx = ((simbad_df["hip"] == 84012) & (simbad_df["componentid"] == 0))
assert np.sum(idx) == 1, "There should be only one HIP 84012"
simbad_df.loc[idx, "componentid"] = 1  # because it is A component
# Turn HIP 36850 to HIP 36850A
idx = (simbad_df["hip"] == 36850) & (simbad_df["componentid"] == 0)
assert np.sum(idx) == 1, "There should be only one HIP 36850"
simbad_df.loc[idx, "componentid"] = 1

# =============================================================================
simbad_df["B_V"] = (simbad_df["B"] - simbad_df["V"]).astype(np.float64).values
v = simbad_df["V"].values
gaia_v_mag = np.zeros(len(simbad_df)) * np.nan
gaia_g_mag = np.zeros(len(simbad_df)) * np.nan
gaia_v_mag[idx2] = gaia_t.loc[idx1, "v_mag"].values
gaia_g_mag[idx2] = gaia_t.loc[idx1, "phot_g_mean_mag"].values
simbad_df.fillna({"V": pd.Series(gaia_v_mag)}, inplace=True)
simbad_df.fillna({"V": pd.Series(gaia_g_mag)}, inplace=True)
# if still missing, fill with J because nothing I can do
simbad_df.fillna({"V": simbad_df["J"]}, inplace=True)
# try to get synthetic photometry
matched_source_id, idx3, idx4 = np.intersect1d(
    simbad_df["source_id"].values,
    synth_phot_df["source_id"].values,
    return_indices=True,
)
# if still missing, fill with J + 3 because nothing I can do
simbad_df.fillna({"V": simbad_df["J"] + 3}, inplace=True)
# if still missing B_V, try inferior B_V estimated from XP spectra
simbad_df.loc[idx3, "B_V"] = (
    simbad_df.loc[idx3, "B_V"]
    .fillna(synth_phot_df.loc[idx4, "Jkc_mag_B"] - synth_phot_df.loc[idx4, "Jkc_mag_V"])
    .values
)
# finally if still missing, estimates from BP-RP
_, _bv = gbprp_to_bv(
    gaia_t.loc[idx1, "phot_g_mean_mag"].values,
    gaia_t.loc[idx1, "bp_rp"].values,
    red_correction=True,
)
simbad_df.loc[idx2, "Gaia_B_V"] = _bv
simbad_df.fillna({"B_V": simbad_df["Gaia_B_V"]}, inplace=True)

# if some stars with componentid >= 2 and still missing V, delete them
simbad_df = simbad_df[~((simbad_df["componentid"] >= 2) & simbad_df["V"].isna())]

# nothing should have 0 vmag, a sign of bad data
assert np.sum(simbad_df["V"] == 0) + np.sum(simbad_df["V"].isna()) == 0, (
    "Some V are still 0 or NaN"
)

# if negative parallax, set to np.nan
neg_parallax_idx = simbad_df["plx_value"] < 0
simbad_df.loc[neg_parallax_idx, "plx_value"] = np.nan
simbad_df.loc[neg_parallax_idx, "plx_err"] = np.nan

df = pd.DataFrame(
    index=range(len(simbad_df)),
    data={
        "hip": simbad_df["hip"].fillna(0).astype(int).values,
        "componentid": simbad_df["componentid"].fillna(0).astype(int).values,
        "source_id": simbad_df["source_id"].fillna(0).astype(np.int64).values,
        "ra": simbad_df["ra"].values,
        "dec": simbad_df["dec"].values,
        "epoch": simbad_df["epoch"].values,
        "parallax": simbad_df["plx_value"].values,
        "parallax_error": simbad_df["plx_err"].values,
        "pmra": simbad_df["pmra"].values,
        "pmdec": simbad_df["pmdec"].values,
        "b_v": simbad_df["B_V"].values,
        "vmag": simbad_df["V"].values,
        "radial_velocity": simbad_df["rvz_radvel"].values,
        "radial_velocity_error": simbad_df["rvz_err"].values,
        "sp_type": simbad_df["sp_type"].values,
        "otype": simbad_df["otype"].values,
    },
)
assert df["source_id"].dtype == np.int64
have_pm = ~np.isnan(df["pmra"]) & ~np.isnan(df["pmdec"])
idx = (df["epoch"] != 2016.0) & have_pm
# those without proper motion, we can't propagate so call it a day
df.loc[~have_pm, "epoch"] = 2016.0

ra, dec, pmra_cosdec, pmdec, parallax, rv = apply_space_motion(
    df.loc[idx, "ra"].to_numpy(),
    df.loc[idx, "dec"].to_numpy(),
    df.loc[idx, "pmra"].to_numpy(),
    df.loc[idx, "pmdec"].to_numpy(),
    parallax=df.loc[idx, "parallax"].fillna(1).to_numpy(),  # just assume at 1 kpc
    rv=df.loc[idx, "radial_velocity"]
    .fillna(0)
    .to_numpy(),  # just assume no radial motion
    t1=Time(df.loc[idx, "epoch"], format="jyear"),
    t2=Time([2016.0] * np.sum(idx), format="jyear"),
)

# only map for those epoch not equal to 2016, because numerical error even J2016 to J2016
df.loc[idx, "ra"] = ra
df.loc[idx, "dec"] = dec
df.loc[idx, "pmra"] = pmra_cosdec
df.loc[idx, "pmdec"] = pmdec
# the effect on parallax and radial velocity within human lifetime is negligible

# =============================================================================
# Process the Gaia DR3 catalog
# =============================================================================
# re-read the Gaia DR3 data, in case the table has been modified
gaia_t = Table.read(gaia_data_path, format="fits")
gaia_t = gaia_t.to_pandas()
_, gaia_t["b_v"] = gbprp_to_bv(
    gaia_t["phot_g_mean_mag"], gaia_t["bp_rp"], red_correction=True
)  # as a base column, in case no synthetic photometry

# if no v_mag, fill with g_mag
gaia_t.fillna({"v_mag": gaia_t["phot_g_mean_mag"]}, inplace=True)

# try to get synthetic photometry
matched_source_id, idx4, idx5 = np.intersect1d(
    gaia_t["source_id"].values, synth_phot_df["source_id"].values, return_indices=True
)
# put the astrometry from Gaia DR3 to SIMBAD, make sure indexing done correctly
synth_b_v = (
    synth_phot_df.loc[idx5, "Jkc_mag_B"].values
    - synth_phot_df.loc[idx5, "Jkc_mag_V"].values
)
gaia_t.loc[idx4, "b_v"] = synth_b_v

# exclude those source_id already included in the simbad_df
gaia_t = gaia_t[~gaia_t["source_id"].isin(simbad_df["source_id"].values)]
good_astrometry_idx = (gaia_t["astrometric_params_solved"].values == 31) | (
    gaia_t["astrometric_params_solved"].values == 95
) & (gaia_t["ruwe"].values < 1.4)

# make df_gaia
df_gaia = pd.DataFrame(
    index=range(len(gaia_t)),
    data={
        "hip": np.zeros(len(gaia_t), dtype=int),
        "componentid": np.zeros(len(gaia_t), dtype=int),
        "source_id": gaia_t["source_id"].astype(np.int64).values,
        "ra": gaia_t["ra"].values,
        "dec": gaia_t["dec"].values,
        "epoch": np.zeros(len(gaia_t), dtype=float)
        + 2016.0,  # we have propagated to J2000.0
        "parallax": gaia_t["parallax"].values,
        "parallax_error": gaia_t["parallax"].values
        / gaia_t["parallax_over_error"].values,
        "pmra": gaia_t["pmra"].values,
        "pmdec": gaia_t["pmdec"].values,
        "b_v": gaia_t["b_v"].values,
        "vmag": gaia_t["v_mag"].values,
        "radial_velocity": gaia_t["radial_velocity"].values,
        "radial_velocity_error": gaia_t["radial_velocity_error"].values,
        "sp_type": [""] * len(gaia_t),
        "otype": ["*"] * len(gaia_t),
    },
)
# nothing should have 0 vmag, a sign of bad data
assert np.sum(df_gaia["vmag"] == 0) == 0
assert df_gaia["source_id"].dtype == np.int64

# combine the two
df = pd.concat([df, df_gaia], ignore_index=True)

# get those still without RA and DEC
source_id_ls = [i for i in df[df["ra"].isnull()]["source_id"].values]
if len(source_id_ls) == 0:
    print("All stars have astrometry")
else:
    # join as one string separated by comma
    source_id_str = "(" + ",".join([str(i) for i in source_id_ls]) + ")"
    print(
        "These source_id is going to be included, exclude these in the next star catalog"
    )
    print(source_id_str)
    job = Gaia.launch_job(f"""SELECT *
                        FROM gaiadr3.gaia_source
                        WHERE source_id in {source_id_str}""")
    jobresult_t = job.results
    jobresult_df = jobresult_t.to_pandas()
    # if there are source_id, use astrometry from Gaia DR3 even if RA/DEC is not NaN
    matched_source_id, idx1, idx2 = np.intersect1d(
        jobresult_df["SOURCE_ID"].values, df["source_id"].values, return_indices=True
    )
    # put the astrometry from Gaia DR3 to SIMBAD, make sure indexing done correctly
    assert np.all(
        df.loc[idx2, "source_id"].values == jobresult_df.loc[idx1, "SOURCE_ID"].values
    )

    # turn to vmag and b_v
    jobresult_t["vmag"], jobresult_t["b_v"] = gbprp_to_bv(
        jobresult_t["phot_g_mean_mag"], jobresult_t["bp_rp"], red_correction=True
    )
    df.loc[idx2, "ra"] = jobresult_df.loc[idx1, "ra"].values
    df.loc[idx2, "dec"] = jobresult_df.loc[idx1, "dec"].values
    df.loc[idx2, "parallax"] = jobresult_df.loc[idx1, "parallax"].values
    df.loc[idx2, "parallax_error"] = (
        jobresult_df.loc[idx1, "parallax"].values
        / jobresult_df.loc[idx1, "parallax_over_error"].values
    )
    df.loc[idx2, "pmra"] = jobresult_df.loc[idx1, "pmra"].values
    df.loc[idx2, "pmdec"] = jobresult_df.loc[idx1, "pmdec"].values
    # df.loc[idx2, "vmag"] = jobresult_df.loc[idx1, "vmag"].values
    # df.loc[idx2, "b_v"] = jobresult_df.loc[idx1, "b_v"].values
    df.loc[idx2, "epoch"] = 2016.0

# RA and DEC should have no NaN
assert df["ra"].isnull().sum() == 0
assert df["dec"].isnull().sum() == 0
# nothing should have 0 vmag, a sign of bad data
assert np.sum(df["vmag"] == 0) + np.sum(df["vmag"].isna()) == 0

# Gaia DR3 1926461164913660160 is Ross 248, will be close to solar neighborhood in the future hence of interest, make sure it is included in the first 3 catalogs
df.loc[df["source_id"] == 1926461164913660160, "hip"] = 9999999
df.loc[df["source_id"] == 1926461164913660160, "otype"] = "BY*"
df.loc[df["source_id"] == 1926461164913660160, "sp_type"] = "M5.0V"
# some stars are missing parallax
df.loc[df["hip"] == 36850, "parallax"] = 64.12
df.loc[df["hip"] == 36850, "parallax_error"] = 3.75
df.loc[df["hip"] == 84345, "parallax"] = 9.07
df.loc[df["hip"] == 84345, "parallax_error"] = 1.32
df.loc[df["hip"] == 110900, "parallax"] = 14.5943
df.loc[df["hip"] == 110900, "parallax_error"] = 0.1562
df.loc[df["hip"] == 54844, "parallax"] = 18.35
df.loc[df["hip"] == 54844, "parallax_error"] = 0.96
df.loc[df["hip"] == 55203, "parallax"] = 114.4867
df.loc[df["hip"] == 55203, "parallax_error"] = 0.4316

# some stars missing astrometry
df.loc[df["source_id"] == 756853643637996160, "parallax"] = 114.4867
df.loc[df["source_id"] == 756853643637996160, "parallax_error"] = 0.4316
df.loc[df["source_id"] == 756853643637996160, "pmra"] = -339.398
df.loc[df["source_id"] == 756853643637996160, "pmdec"] = -607.892
df.loc[df["source_id"] == 756853643637996160, "radial_velocity"] = -15.9
df.loc[df["source_id"] == 756853643637996160, "radial_velocity_error"] = 0.9
df.loc[df["source_id"] == 756853643637996160, "sp_type"] = "G2V"
df.loc[df["source_id"] == 756853643637996160, "b_v"] = 5.41 - 4.77

df.loc[df["source_id"] == 3683687763520080256, "parallax"] = 78.5233
df.loc[df["source_id"] == 3683687763520080256, "parallax_error"] = 1.3879
df.loc[df["source_id"] == 3683687763520080256, "pmra"] = -534.318
df.loc[df["source_id"] == 3683687763520080256, "pmdec"] = -64.270
df.loc[df["source_id"] == 3683687763520080256, "radial_velocity"] = -19.8
df.loc[df["source_id"] == 3683687763520080256, "radial_velocity_error"] = 0.9
df.loc[df["source_id"] == 3683687763520080256, "sp_type"] = "F0mF2V"
df.loc[df["source_id"] == 3683687763520080256, "b_v"] = 3.85 - 3.49

# other columns should not have NaN, if yes then something is wrong
df = df.fillna(
    {
        "b_v": 0.65,
        "parallax": 0,
        "parallax_error": 0,
        "pmra": 0,
        "pmdec": 0,
        "radial_velocity": 0,
        "radial_velocity_error": 0,
        "sp_type": "",
        "otype": "*",
    }
)
assert df["source_id"].dtype == np.int64
neg_parallax_idx = df["parallax"] < 0
df.loc[neg_parallax_idx, "parallax"] = 0
df.loc[neg_parallax_idx, "parallax_error"] = 0

# should not be a problem anymore
# # find duplicated source_id, drop the duplicates that have hip_id = 0
# # this is caused by SIMBAD resolving the Gaia-HIP cross match but not in official Gaia release
# df.drop(
#     df[df.duplicated(subset="source_id", keep=False)]
#     .query("source_id > 0 and hip == 0")
#     .index,
#     inplace=True,
# )

In [ ]:
warnings.filterwarnings("ignore", module="erfa")

min_vmag = [-2.0, 6.0, 7.5, 9.0, 10.5, 12.0, 13.75]
max_vmag = [6.0, 7.5, 9.0, 10.5, 12.0, 13.75, 15.5]
major_version = [0, 0, 0, 0, 0, 0, 0]
minor_version = [20, 16, 17, 10, 6, 6, 4]
data_type = [0, 0, 0, 0, 1, 1, 1]
level = [0, 1, 2, 3, 4, 5, 6]
# J2000.0 is 2451545.0
# J2016.0 is 2457389.0
catalog_epoch_jd = Time(2016, format="jyear").jd

# =============================================================================
# Write to file
# =============================================================================

# count all spectral type
spectral_type_ls = []
new_spectral_type_flag = True  # flag to indicate if we are creating a new spectral type
spectral_type_file_name = starcatalog_base_path / "stars_hip_sp_0v0_6.cat"
if spectral_type_file_name.exists():
    new_spectral_type_flag = False
    with open(spectral_type_file_name, "r") as f:
        for line in f:
            spectral_type_ls.append(line.strip())

otype_ls = []
# read otype file
with open(starcatalog_base_path / "otype.dat", "r") as f:
    for line in f:
        otype_ls.append(line.strip())


def encode_star_hip(hip, letter_value):
    # Ensure hip is between 0 and 120000 (17-bit)
    if (
        hip == 9999999
    ):  # some hip stars are filled with 9999999 to be incldued in the first 3 catalogs
        hip = 0
    if hip < 0 or hip > 2**17:
        raise ValueError(f"HIP must be between 0 and {2**17}.")

    # Combine the 17-bit ID and the 5-bit letter value into a 24-bit integer
    combined_value = (hip << 5) | letter_value

    # Pack the 24-bit value into 3 bytes
    return struct.pack("<I", combined_value)[:3]


for lv, datatype, majver, minver, min_v, max_v in zip(
    level,
    data_type,
    major_version,
    minor_version,
    min_vmag,
    max_vmag,
):
    print(lv)
    if lv == 0 or lv == 1:
        df6 = df[(df["vmag"] > min_v) & (df["vmag"] <= max_v)].reset_index(drop=True)
    elif lv == 2:  # level 2 is special
        # on top of the vmag range, include everything with a hip id
        df6 = df[
            ((df["vmag"] > min_v) & (df["hip"] > 0))
            | (df["vmag"] > min_v) & (df["vmag"] <= max_v)
        ].reset_index(drop=True)
    else:  # other level should only include stars with NaN hip id
        df6 = df[
            (df["vmag"] > min_v) & (df["vmag"] <= max_v) & (df["hip"] == 0)
        ].reset_index(drop=True)

    # =============================================================================
    # Deal with geodesic grid
    # =============================================================================
    grid = GeodesicGrid(level=lv)

    # setup ray-tracing
    xyz = radec2xyz(df6["ra"], df6["dec"]).T
    vertices = np.vstack(grid.vertices[-1])
    faces = np.vstack(grid.faces[-1])
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    ray_origins = np.tile([[0, 0, 0]], (len(df6), 1))
    ray_directions = np.array(xyz)
    intersected_faces, ray_indices = mesh.ray.intersects_id(
        ray_origins=ray_origins,
        ray_directions=ray_directions,
        multiple_hits=False,
    )
    df6.loc[ray_indices, f"zone{lv}"] = intersected_faces
    # find which number ray_indices are missing
    missing = np.setdiff1d(np.arange(len(df6)), ray_indices)
    # if there are missing values, we need to do a more expensive search with algebra
    zones = grid.search_zone(xyz[missing])
    df6.loc[missing, f"zone{lv}"] = zones

    # =============================================================================
    # Deal with past and future zones
    # =============================================================================
    if lv == 0 or lv == 1 or lv == 2:
        c = SkyCoord(
            ra=df6["ra"].values * u.deg,
            dec=df6["dec"].values * u.deg,
            distance=df6[["parallax"]].apply(lambda x: 1. / x)["parallax"].values * u.kpc,
            pm_ra_cosdec=df6["pmra"].values * u.mas / u.yr,
            pm_dec=df6["pmdec"].values * u.mas / u.yr,
            radial_velocity=df6["radial_velocity"].values * u.km / u.s,
            obstime=Time(2016.0, format="jyear"),
            frame="icrs",
        )
        g = GeodesicGrid(level=lv)
        zone_now = g.search_zone(c.cartesian.xyz.T)
        zone_past = g.search_zone(c.cartesian.xyz.T)
        zone_future = g.search_zone(c.cartesian.xyz.T)
        good_astrometry_idx = df6["parallax"] / df6["parallax_error"] > 5

        # calculate absmag now
        absmag = df6["vmag"] - 5 * np.log10(1000.0 / df6["parallax"]) + 5

        counter_past = np.zeros(len(c))
        counter_future = np.zeros(len(c))
        max_vmag_diff = np.zeros(len(c))

        for iyr in range(10000, 210000, 10000):  # over 10000 years to have some buffer
            c_past = SkyCoord(
                ra=df6["ra"].values * u.deg,
                dec=df6["dec"].values * u.deg,
                distance=df6[["parallax"]].apply(lambda x: 1. / x)["parallax"].values * u.kpc,
                pm_ra_cosdec=df6["pmra"].values * u.mas / u.yr,
                pm_dec=df6["pmdec"].values * u.mas / u.yr,
                radial_velocity=df6["radial_velocity"].values * u.km / u.s,
                obstime=Time(iyr, format="jyear"),
                frame="icrs",
            )
            c_past = c_past.apply_space_motion(Time(0.0, format="jyear"))
            temp_zone = g.search_zone(c_past.cartesian.xyz.T)
            counter_past += temp_zone != zone_past
            zone_past = temp_zone

            c_future = SkyCoord(
                ra=df6["ra"].values * u.deg,
                dec=df6["dec"].values * u.deg,
                distance=df6[["parallax"]].apply(lambda x: 1. / x)["parallax"].values * u.kpc,
                pm_ra_cosdec=df6["pmra"].values * u.mas / u.yr,
                pm_dec=df6["pmdec"].values * u.mas / u.yr,
                radial_velocity=df6["radial_velocity"].values * u.km / u.s,
                obstime=Time(0.0, format="jyear"),
                frame="icrs",
            )
            c_future = c_future.apply_space_motion(Time(iyr, format="jyear"))
            temp_zone = g.search_zone(c_future.cartesian.xyz.T)
            counter_future += temp_zone != zone_future
            zone_future = temp_zone

            # calculate vmag in the future and the past
            # need to check in a for loop because some stars get bright and dim again
            vmag_past = 5 * np.log10(c_future.distance.value) - 5 + absmag
            vmag_future = 5 * np.log10(c_past.distance.value) - 5 + absmag
            max_vmag_diff = np.min(
                [max_vmag_diff, vmag_past - df6["vmag"], vmag_future - df6["vmag"]],
                axis=0,
            )

        max_vmag_diff[np.isinf(max_vmag_diff) | ~good_astrometry_idx] = 0
        if (
            lv == 0
        ):  # for level 0, we are more lenient because easy to zoom enough that border triangles are not checked
            # no need to check brightness because these stars are very bright already
            total_counter = (counter_past + counter_future > 0) & good_astrometry_idx
        else:
            total_counter = np.logical_or(
                np.logical_or(counter_past > 1, counter_future > 1),
                max_vmag_diff < -0.3,
            )
        print(np.sum(total_counter))
        df6.loc[total_counter, f"zone{lv}"] = 20 * 4**lv

    # sort by zone, within each zone sort by vmag
    df6 = df6.sort_values([f"zone{lv}", "vmag"]).reset_index(drop=True)
    df6["pmra_wo_cosdec"] = df6["pmra"] / np.cos(np.radians(df6["dec"]))

    f = open(
        starcatalog_base_path / f"./stars_{lv}_{datatype}v{majver}_{minver}.cat", "w+b"
    )

    f.write(b"\n\x04_\x83")  # Magic Number
    f.write(np.int32(datatype).tobytes())  # Data Type
    f.write(np.int32(majver).tobytes())  # Major Version
    f.write(np.int32(minver).tobytes())  # Minor Version
    f.write(np.int32(lv).tobytes())  # Level
    f.write(np.int32(min_v * 1000).tobytes())  # Magnitude Minimum
    f.write(np.float32(catalog_epoch_jd).tobytes())  # Catalog Epoch
    n_zones = 20 * 4**lv + 1  # plus 1 global zone

    # count number of stars in each zone in df6
    zone_info = df6[f"zone{lv}"].value_counts().sort_index()
    for z in range(n_zones):
        f.write(struct.pack("I", zone_info.get(z, 0)))

    max_records = sum(zone_info)

    df6["source_id"] = df6["source_id"].astype(np.int64)
    if datatype == 0:
        # =============================================================================
        # astrometry
        # =============================================================================
        sra = np.sin(np.radians(df6["ra"]))
        cra = np.cos(np.radians(df6["ra"]))
        sdec = np.sin(np.radians(df6["dec"]))
        cdec = np.cos(np.radians(df6["dec"]))
        # normal triad
        p = np.array([-sra, cra, np.zeros(len(df6))])
        q = np.array([-sdec * cra, -sdec * sra, cdec])
        r = np.array([cdec * cra, cdec * sra, sdec])
        # proper motion
        pmvec0 = np.atleast_2d(df6["pmra"]) * p + np.atleast_2d(df6["pmdec"]) * q

        r *= 2e9
        pmvec0 *= 1000
        # round to nearest integer
        r = np.around(r).astype(np.int32)
        pmvec0 = np.around(pmvec0).astype(np.int32)
        df6["b_v_1000"] = (df6["b_v"] * 1000.0).round().astype(int)
        df6["vmag_1000"] = (df6["vmag"] * 1000.0).round().astype(int)
        df6["parallax_50"] = (df6["parallax"] * 50.0).round().astype(int)
        df6["parallax_error_100"] = (df6["parallax_error"] * 100.0).round().astype(int)
        # if radial velocity is rudiculously large, set to 0 (suspicious e.g., a quasar)
        df6["radial_velocity_10"] = (
            np.where(
                np.abs(df6["radial_velocity"]) > np.iinfo(np.int16).max / 10.0,
                0,
                df6["radial_velocity"] * 10,
            )
            .round()
            .astype(int)
        )

        # pre-allocate
        bdata = bytearray(48 * max_records)

        for i, row in df6.iterrows():
            # star_header = (
            #     "gaia_id",
            #     "x0"
            #     "x1"
            #     "x2",
            #     "dx0",
            #     "dx1",
            #     "dx2",
            #     "b_v",
            #     "mag",
            #     "plx",
            #     "plx_err",
            #     "rv"
            #     "sp_int",
            #     "object_type",
            #     "hip + component",
            # )
            _sptype_index = 0
            _sptype = row["sp_type"]
            if (
                _sptype not in spectral_type_ls
                and _sptype != ""
                and new_spectral_type_flag
            ):
                spectral_type_ls.append(_sptype)
            try:
                _sptype_index = spectral_type_ls.index(_sptype) + (
                    1 * new_spectral_type_flag
                )
            except ValueError:
                pass  # it can be the case where we are not creating a new spectral type files but the spectral type is not in the list

            hip_bytes = encode_star_hip(np.uint32(row["hip"]), int(row["componentid"]))
            byte_data = (
                struct.pack(
                    "qiiiiiihhHHhHB",
                    row["source_id"],
                    r[0, i],
                    r[1, i],
                    r[2, i],
                    pmvec0[0, i],
                    pmvec0[1, i],
                    pmvec0[2, i],
                    row["b_v_1000"],
                    row["vmag_1000"],
                    row["parallax_50"],
                    row["parallax_error_100"],
                    row["radial_velocity_10"],
                    _sptype_index,  # 0 is reserved for No Information
                    otype_ls.index(row["otype"]),
                )
                + hip_bytes
            )
            bdata[i * 48 : (1 + i) * 48] = byte_data
    elif datatype == 1:
        # round to nearest integer
        df6["ra_3600000"] = (df6["ra"] * 3_600_000).round().astype(int)
        df6["dec_3600000"] = (df6["dec"] * 3_600_000).round().astype(int)
        df6["pmra_wo_cosdec_1000"] = (df6["pmra_wo_cosdec"] * 1000).round().astype(int)
        df6["pmdec_1000"] = (df6["pmdec"] * 1000).round().astype(int)
        df6["b_v_1000"] = (df6["b_v"] * 1000).round().astype(int)
        df6["vmag_1000"] = (df6["vmag"] * 1000).round().astype(int)
        df6["parallax_100"] = (df6["parallax"] * 100).round().astype(int)
        df6["parallax_error_100"] = (df6["parallax_error"] * 100).round().astype(int)
        bdata = bytearray(32 * max_records)

        # def pack_star_data(row):
        #     return struct.pack(
        #     "qiiiihhHH",
        #     row["source_id"],
        #     row["ra_3600000"],
        #     row["dec_3600000"],
        #     row["pmra_wo_cosdec_1000"],
        #     row["pmdec_1000"],
        #     row["b_v_1000"],
        #     row["vmag_1000"],
        #     row["parallax_100"],
        #     row["parallax_error_100"],
        #     )

        # bdata = bytearray(df6.apply(pack_star_data, axis=1).sum())

        for i, row in df6.iterrows():
            # star_header = (
            #     "gaia_id",
            #     "x0",
            #     "x1",
            #     "dx0",
            #     "dx1",
            #     "b_v",
            #     "mag",
            #     "plx",
            #     "plx_err"
            # )
            byte_data = struct.pack(
                "qiiiihhHH",
                row["source_id"],
                row["ra_3600000"],
                row["dec_3600000"],
                row["pmra_wo_cosdec_1000"],
                row["pmdec_1000"],
                row["b_v_1000"],
                row["vmag_1000"],
                row[
                    "parallax_100"
                ],  # switch form 50 to 100 beacuse proxima centauri is not here
                row["parallax_error_100"],
            )
            bdata[i * 32 : (1 + i) * 32] = byte_data
    else:
        raise ValueError("Data Type must be 0 or 1")
    f.write(bdata)
    f.close()

# write the spectral type to a file, first line empty, then one spectral type per line
if new_spectral_type_flag:
    with open(spectral_type_file_name, "wb") as f:
        f.write(b"\n")
        for s in spectral_type_ls:
            f.write(f"{s}\n".encode())